### Build a Simple LLM Application with LCEL
In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!

After seeing this video, you'll have a high level overview of:

- Using language models

- Using PromptTemplates and OutputParsers

- Using LangChain Expression Language (LCEL) to chain components together

- Debugging and tracing your application using LangSmith

- Deploying your application with LangServe

In [1]:
### Open AI API Key and Open Source models--Llama3,Gemma2,mistral--Groq

import os
from dotenv import load_dotenv
load_dotenv()

import openai
openai.api_key=os.getenv("OPENAI_API_KEY")

groq_api_key=os.getenv("GROQ_API_KEY")
groq_api_key

'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'

In [2]:
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000023300411E10>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000233004129E0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [5]:
from langchain_core.messages import HumanMessage,SystemMessage

messages=[
    SystemMessage(content="Translate the following from English to Spanish"),
    HumanMessage(content="Hello How are you?")
]

result=model.invoke(messages)

In [6]:
result

AIMessage(content="Here's the translation:\n\n* **Hello:** Hola \n* **How are you?:** ¿Cómo estás? (informal, to one person) or ¿Cómo están? (formal, to multiple people) \n\n\nLet me know if you have any other phrases you'd like translated!\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 65, 'prompt_tokens': 21, 'total_tokens': 86, 'completion_time': 0.118181818, 'prompt_time': 0.000149609, 'queue_time': 0.02027119, 'total_time': 0.118331427}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-f349ccd8-ece2-4580-87f1-828a055f822d-0', usage_metadata={'input_tokens': 21, 'output_tokens': 65, 'total_tokens': 86})

In [12]:
from langchain_core.output_parsers import StrOutputParser
parser=StrOutputParser()
parser.invoke(result)

"Here's the translation:\n\n* **Hello:** Hola \n* **How are you?:** ¿Cómo estás? (informal, to one person) or ¿Cómo están? (formal, to multiple people) \n\n\nLet me know if you have any other phrases you'd like translated!\n"

In [13]:
### Using LCEL- chain the components
chain = model | parser
chain.invoke(messages)

'Here are a few ways to say "Hello, how are you?" in Spanish:\n\n**Formal:**\n\n* **Hola, ¿cómo está?** (This is used when addressing someone you don\'t know well or someone older than you.)\n\n**Informal:**\n\n* **Hola, ¿cómo estás?** (This is used when addressing someone you know well, like a friend or family member.)\n\n\nLet me know if you\'d like to know other ways to greet someone in Spanish! 😊\n'

In [14]:
### Prompt Templates
from langchain_core.prompts import ChatPromptTemplate

generic_template="Tranaslate the following into {language}:"

prompt=ChatPromptTemplate.from_messages(
    [("system",generic_template),("user","{text}")]
)



In [15]:
result=prompt.invoke({"language":"Spanish","text":"Hello"})

In [16]:
result.to_messages()

[SystemMessage(content='Tranaslate the following into Spanish:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello', additional_kwargs={}, response_metadata={})]

In [18]:
##Chaining together components with LCEL
chain=prompt|model|parser
chain.invoke({"language":"Spanish","text":"Hello"})

'Hola \n'